# Transaction Build - BX Example
In this BX tutorial, we construct a transaction which spends one P2PKH output to six other outputs.

<img src="images/first_transaction.jpg" alt="drawing" style="" width="700px"/>



<hr style="border: 0.5px dashed #000;">

## 1. Setup wallet and test funds.

We will be spending from a funded testnet address to other testnet address derived from the same hd-wallet, so that we can reuse these coins in later exercises. For simplicity purposes, we will derive all keys from the same mnemonic word list generated here.

### Generate mnemonic and hd master keys.

In [ ]:
bx seed | bx ec-new

In [ ]:
bx seed --bit_length 128 | bx mnemonic-new --language en

In [ ]:
my_mnemonic="amount right cheese defy click eight slight strategy replace earn simple labor"
hd_master_private=$(bx mnemonic-to-seed $my_mnemonic | bx hd-new)
hd_master_public=$(bx hd-to-public $hd_master_private)


### Generate spending key `m/44'/1'/0'/0/0`.


In [ ]:
# We will send funds to keys from account 0.

# Account 0 receiving parent: m/44'/1'/0'/0
hd_m_44h_1h_0h_0=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 0  \
| bx hd-private --index 0)

# Test funds will be sent from: m/44'/1'/0'/0/0
hd_m_44h_1h_0h_0_0=$(bx hd-private --index 0 $hd_m_44h_1h_0h_0)
privatekey_44h_1h_0h_0_0=$(bx hd-to-ec $hd_m_44h_1h_0h_0_0)
publickey_44h_1h_0h_0_0=$(bx hd-to-public $hd_m_44h_1h_0h_0_0 | bx hd-to-ec)
publickeyhash_44h_1h_0h_0_0=$(bx sha256 $publickey_44h_1h_0h_0_0 | bx ripemd160)
address_44h_1h_0h_0_0=$(bx hd-to-public $hd_m_44h_1h_0h_0_0 | bx hd-to-ec | bx ec-to-address --version 111)

echo $address_44h_1h_0h_0_0
echo $publickeyhash_44h_1h_0h_0_0


### Send test coins funds to keys ( `/44'/1'/0'/0/0` ) you will be spending from.
Note your spendable p2pkh UXTO's below...

In [5]:
previous_txid=bc58d14fefb5a0f2de565cfa184175e9fc9a584bdb9c4b1deba6965580e0c33c
previous_output_index=1
previous_output_amount=11790043


### Generate destination key pairs.

We will spend the outputs sent to these addresses in later examples & exercises.<br>
We need to derive public key hash for the respective output script.


In [21]:
# We will send funds to keys in account 1.

# Account 1 receiving parent: m/44'/1'/1'/0
hd_m_44h_1h_1h_0=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 1  \
| bx hd-private --index 0)

hd_m_44h_1h_1h_0_1=$(bx hd-private --index 1 $hd_m_44h_1h_1h_0)
publickeyhash_44h_1h_1h_0_1=$(bx hd-to-public $hd_m_44h_1h_1h_0_1 | bx hd-to-ec | bx sha256 | bx ripemd160)

hd_m_44h_1h_1h_0_2=$(bx hd-private --index 2 $hd_m_44h_1h_1h_0)
publickeyhash_44h_1h_1h_0_2=$(bx hd-to-public $hd_m_44h_1h_1h_0_2 | bx hd-to-ec | bx sha256 | bx ripemd160)

hd_m_44h_1h_1h_0_3=$(bx hd-private --index 3 $hd_m_44h_1h_1h_0)
publickeyhash_44h_1h_1h_0_3=$(bx hd-to-public $hd_m_44h_1h_1h_0_3 | bx hd-to-ec | bx sha256 | bx ripemd160)

hd_m_44h_1h_1h_0_4=$(bx hd-private --index 4 $hd_m_44h_1h_1h_0)
publickeyhash_44h_1h_1h_0_4=$(bx hd-to-public $hd_m_44h_1h_1h_0_4 | bx hd-to-ec | bx sha256 | bx ripemd160)

hd_m_44h_1h_1h_0_5=$(bx hd-private --index 5 $hd_m_44h_1h_1h_0)
publickeyhash_44h_1h_1h_0_5=$(bx hd-to-public $hd_m_44h_1h_1h_0_5 | bx hd-to-ec | bx sha256 | bx ripemd160)

hd_m_44h_1h_1h_0_6=$(bx hd-private --index 6 $hd_m_44h_1h_1h_0)
publickeyhash_44h_1h_1h_0_6=$(bx hd-to-public $hd_m_44h_1h_1h_0_6 | bx hd-to-ec | bx sha256 | bx ripemd160)


## 2. Spending a P2PKH output.

### Create output scripts.

* **Compose P2PKH(destination public key hash) output scripts for each destination.**

In [9]:
# bx script-encode "DUP HASH160 [public key hash] EQUALVERIFY CHECKSIG"
output_script1=$(bx script-encode "DUP HASH160 ["$publickeyhash_44h_1h_1h_0_1"] EQUALVERIFY CHECKSIG")
output_script2=$(bx script-encode "DUP HASH160 ["$publickeyhash_44h_1h_1h_0_2"] EQUALVERIFY CHECKSIG")
output_script3=$(bx script-encode "DUP HASH160 ["$publickeyhash_44h_1h_1h_0_3"] EQUALVERIFY CHECKSIG")
output_script4=$(bx script-encode "DUP HASH160 ["$publickeyhash_44h_1h_1h_0_4"] EQUALVERIFY CHECKSIG")
output_script5=$(bx script-encode "DUP HASH160 ["$publickeyhash_44h_1h_1h_0_5"] EQUALVERIFY CHECKSIG")
output_script6=$(bx script-encode "DUP HASH160 ["$publickeyhash_44h_1h_1h_0_6"] EQUALVERIFY CHECKSIG")
echo $output_script6


76a9145f77593cbeae0845c6843cc05d9cfb1b8d959de588ac


### Compute output amount(s).

* **Subtract minimum fee, which consists of:**
    * Example minimum fee levels: `100 sat / sigop + 1 sat / byte`
    * This is node policy. 


* **Transaction sigop count computation:**
    * Counted as 1 sigop: `checksig`,`checksigverify`
    * Counted as 20 sigops: `checkmultisig`, `checkmultisigverify`
    * *Non-segwit:*
        * Transaction sigop count is multiplied by 4x.
        * Output vs Input sigops: 
            * All sigops in the output script are counted.
            * P2SH Redeemscript input sigops are counted.
      

In [ ]:
# Sigops: 6 outputs, so total of 6 sigops.
# tx bytes: 4 + 1 + 1*(32+4+110+4) + 1 + 6*(8+33) + 4 
tx_byte_count=$(expr 4 + 1 + 1 \* 150 + 1 + 6 \* 41 + 4)
echo $tx_byte_count


In [13]:
# Output amount total.
tx_total_output_amount=$(expr $previous_output_amount - 6 \* 400 - $tx_byte_count \* 1)


In [14]:
# spent amount evenly across outputs.
single_output_amount=$(expr $tx_total_output_amount / 6)
echo $single_output_amount


1964541


### Construct transaction template for signing.

* `sequence: 0xffffffff(hex)/4294967295(dec)`

In [ ]:
# bx tx-encode \
# --input [previous tx id]:[index]:[sequence] \
# --output [output script]:[output amount]

my_tx=$(bx tx-encode \
--input $previous_txid:$previous_output_index:4294967295 \
--output $output_script1:$single_output_amount \
--output $output_script2:$single_output_amount \
--output $output_script3:$single_output_amount \
--output $output_script4:$single_output_amount \
--output $output_script5:$single_output_amount \
--output $output_script6:$single_output_amount)


### Sign transaction.

**Fetch previous output script**

In [16]:
# bx fetch-tx --format json [previous tx id] 
previous_output_script=$(bx fetch-tx --format json $previous_txid \
| jq -r ".transaction.outputs[1].script")

echo $previous_output_script

dup hash160 [e48199d47742b245464b1366d95ef26aa4c8bb2c] equalverify checksig


**Sign transaction with previous output script.**

In [17]:
# bx input-sign [private key] "previous output script" [transaction template]
signature=$(bx input-sign --sign_type all --index 0 $privatekey_44h_1h_0h_0_0 "$previous_output_script" $my_tx)


**Set the input script into the finalised transaction.**

In [ ]:
# bx input-set "[signature] [public key point]" [transaction template]

my_tx=$(bx input-set --index 0 "[$signature] [$publickey_44h_1h_0h_0_0]" $my_tx)


## 3. Validate & Broadcast the endorsed transaction

In [24]:
bx validate-tx $my_tx

The transaction is valid.


In [25]:
bx send-tx $my_tx

Sent transaction.


In [26]:
# bx bitcoin256 [tx data]
bx bitcoin256 $my_tx


d4e68f7e163de4c31c6d4d75a38daa015a968371c1a94e2a182e29002235a168
